In [ ]:
import random
import string
import json
from datetime import datetime
from collections import defaultdict

# Conjunto para almacenar los códigos generados
codigos_generados = set()

# Función para generar un código aleatorio único de 6 caracteres
def generar_codigo_aleatorio_unico():
    while True:
        codigo = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
        if codigo not in codigos_generados:  # Verificar si el código ya existe
            codigos_generados.add(codigo) 
            return codigo 

# Clase Producto
class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, tipo=None, cantidad=0, precio=0):
        self.codigo = generar_codigo_aleatorio_unico()  
        self.nombre = nombre
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.tipo = tipo  # Carnes, Frutas, Cereales, Conservas, Lácteos
        self.cantidad = int(cantidad) 
        self.precio = float(precio)  
        self.siguiente = None  

    @property
    def precio_total(self):
        return self.precio * self.cantidad 

# Clase Nodo
class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo
        self.producto = None
        self.hijo = None  # Primer producto
        self.hermano = None  # Siguiente nodo del mismo nivel

    def agregar_producto(self, nuevo_producto):
        if not self.producto:
            self.producto = nuevo_producto
        else:
            actual = self.producto
            anterior = None
            while actual:
                anterior = actual
                actual = actual.siguiente
            nuevo_producto.siguiente = actual
            if anterior:
                anterior.siguiente = nuevo_producto
            else:
                self.producto = nuevo_producto

    def agregar_hijo(self, nodo_hijo):
        if not self.hijo:
            self.hijo = nodo_hijo
        else:
            actual = self.hijo
            while actual.hermano:
                actual = actual.hermano
            actual.hermano = nodo_hijo

    def buscar_producto_por_tipo(self, tipo):
        if self.tipo == tipo:
            return self
        actual = self.hijo
        while actual:
            resultado = actual.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
            actual = actual.hermano
        return None

    def buscar_producto_por_vencimiento(self, fecha):
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        producto_actual = self.producto
        while producto_actual:
            if producto_actual.fecha_vencimiento <= fecha_busqueda:
                print(f"Producto vencido: {producto_actual.nombre}")
            producto_actual = producto_actual.siguiente
        hijo = self.hijo
        while hijo:
            hijo.buscar_producto_por_vencimiento(fecha)
            hijo = hijo.hermano

# Clase Arbol
class Arbol:
    def __init__(self, tipo_raiz):
        self.raiz = Nodo(tipo_raiz)
        self.contador_productos = 0 
        self.limite_productos = 10000 
        self.tipos_validos = ["Carnes", "Frutas", "Lácteos", "Cereales", "Conservas", "Vegetales"]  
        
        # Definir los tipos por defecto en el árbol
        for tipo in self.tipos_validos:
            nodo_tipo = Nodo(tipo)
            self.raiz.agregar_hijo(nodo_tipo)

    def agregar_tipo(self, nodo_tipo):
        self.raiz.agregar_hijo(nodo_tipo)

    def agregar_producto_a_tipo(self, tipo, producto):
        if tipo not in self.tipos_validos:
            print(f"Error: El tipo de producto '{tipo}' no es válido. Los tipos válidos son {', '.join(self.tipos_validos)}.")
            return

        if self.contador_productos >= self.limite_productos:
            print(f"Limite de productos alcanzado. Actualmente hay {self.contador_productos} productos en el almacén.")
            return

        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        if nodo_tipo:
            nodo_tipo.agregar_producto(producto)
            self.contador_productos += 1
        else:
            print(f"Tipo de producto '{tipo}' no encontrado.")
    
    def buscar_por_tipo(self, tipo):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        return nodo_tipo.producto if nodo_tipo else None

    def buscar_por_vencimiento(self, fecha):
        self.raiz.buscar_producto_por_vencimiento(fecha)

    def calcular_suma_cantidad_por_tipo(self):
        cantidades = defaultdict(int)
        self._calcular_suma_cantidad_por_tipo_recursivo(self.raiz, cantidades)
        return dict(cantidades)

    def _calcular_suma_cantidad_por_tipo_recursivo(self, nodo, cantidades):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidades[nodo.tipo] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hijo, cantidades)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hermano, cantidades)

    def calcular_suma_cantidad_por_nombre(self):
        nombres = defaultdict(int)
        self._calcular_suma_cantidad_por_nombre_recursivo(self.raiz, nombres)
        return dict(nombres)

    def _calcular_suma_cantidad_por_nombre_recursivo(self, nodo, nombres):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                nombres[producto_actual.nombre] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hijo, nombres)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hermano, nombres)

    def calcular_suma_cantidad_por_tipo_y_precio(self):
        tipo_precio = defaultdict(float)
        self._calcular_suma_cantidad_por_tipo_y_precio_recursivo(self.raiz, tipo_precio)
        return dict(tipo_precio)

    def _calcular_suma_cantidad_por_tipo_y_precio_recursivo(self, nodo, tipo_precio):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                tipo_precio[nodo.tipo] += producto_actual.precio_total
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_y_precio_recursivo(nodo.hijo, tipo_precio)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_y_precio_recursivo(nodo.hermano, tipo_precio)


In [2]:
def imprimir_productos_por_tipo(arbol, tipo):
    nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
    if nodo_tipo:
        print(f"Productos de tipo '{tipo}':")
        producto_actual = nodo_tipo.producto
        while producto_actual:
            # Mostrar el código junto con los demás detalles
            print(f"Código: {producto_actual.codigo}, Nombre: {producto_actual.nombre}, Vencimiento: {producto_actual.fecha_vencimiento}, Cantidad: {producto_actual.cantidad}, Precio unitario: {producto_actual.precio}, Precio total: {producto_actual.precio_total}")
            producto_actual = producto_actual.siguiente
    else:
        print(f"Tipo de producto '{tipo}' no encontrado.")

In [ ]:
# Función para cargar datos desde archivo
def cargar_datos_desde_archivo(nombre_archivo, arbol):
    with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
        lineas = archivo.readlines()
        for linea in lineas:
            datos = json.loads(linea.strip())
            producto = Producto(
                nombre=datos["nombre"],
                fecha_vencimiento=datos.get("fech_venc"),
                tipo=datos["tipo"],  # Carnes, Frutas, Cereales, Conservas, Lácteos
                cantidad=int(datos["cantidad"]), 
                precio=float(datos["precio"]) 
            )
            # Verificar si el tipo de producto ya existe, y si no, agregarlo
            nodo_tipo = arbol.raiz.buscar_producto_por_tipo(producto.tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(producto.tipo)
                arbol.agregar_tipo(nodo_tipo)
            arbol.agregar_producto_a_tipo(producto.tipo, producto)

In [ ]:
# Método para contar todos los productos en el árbol
def contar_productos(arbol):
    cantidad_total = 0
    
    # Recorrer recursivamente el árbol y contar los productos
    def contar_nodo(nodo):
        nonlocal cantidad_total
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidad_total += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            contar_nodo(nodo.hijo)
        if nodo.hermano:
            contar_nodo(nodo.hermano)
    
    contar_nodo(arbol.raiz)
    return cantidad_total

# Método para agregar un producto
def agregar_producto(arbol, nombre, fecha_vencimiento, tipo, cantidad, precio):
    # Contar los productos actuales en el árbol
    cantidad_actual = contar_productos(arbol)
    
    # Verificar si no se ha excedido el límite de productos
    if cantidad_actual + cantidad > 10000:
        print(f"El límite de productos ({cantidad_actual}) ha sido alcanzado. No se puede agregar más productos.")
        return
    
    # Crear el producto a partir de los datos proporcionados
    producto = Producto(
        nombre=nombre,
        fecha_vencimiento=fecha_vencimiento,
        tipo=tipo,
        cantidad=int(cantidad),  
        precio=float(precio) 
    )
    
    # Verificar si el tipo de producto ya existe en el árbol
    nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
    
    # Si el nodo del tipo no existe, crear un nuevo nodo
    if not nodo_tipo:
        nodo_tipo = Nodo(tipo)
        arbol.agregar_tipo(nodo_tipo)
    
    # Agregar el producto al tipo correspondiente
    arbol.agregar_producto_a_tipo(tipo, producto)
    print(f"Producto '{nombre}' agregado correctamente.")


In [ ]:
from collections import deque
from datetime import datetime

class Cola:
    def __init__(self):
        self.cola = deque()
        self.cola_vencidos = deque()  

    def agregar(self, lote_producto):
        """Agregar un lote de producto a la cola"""
        self.cola.append(lote_producto)

    def agregar_desde_arbol(self, arbol, tipo_producto):
        """Agregar productos de un tipo específico desde el árbol a la cola"""
        nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo_producto)
        if nodo_tipo:
            producto_actual = nodo_tipo.producto
            while producto_actual:
                # Agregar el producto actual a la cola con la información necesaria
                self.agregar({
                    'codigo': producto_actual.codigo,  
                    'nombre': producto_actual.nombre,
                    'fech_venc': producto_actual.fecha_vencimiento.strftime("%Y-%m-%d"),  # Convertir fecha a formato string
                    'cantidad': producto_actual.cantidad,
                })
                producto_actual = producto_actual.siguiente
            print(f"Productos del tipo '{tipo_producto}' agregados a la cola.")
        else:
            print(f"Error: El tipo de producto '{tipo_producto}' no existe en el árbol.")

    def retirar(self):
        """Retirar productos de la cola y verificar vencimiento"""
        if len(self.cola) > 0:
            productos_no_vencidos = []
            while len(self.cola) > 0:
                lote = self.cola.popleft()  # Sacar el primer lote de la cola
                if self._producto_vencido(lote):
                    print(f"Lote de {lote['nombre']} ({lote['codigo']}) eliminado debido a su fecha de vencimiento.")
                    self.cola_vencidos.append(lote)
                else:
                    productos_no_vencidos.append(lote)
            for lote in productos_no_vencidos:
                self.cola.append(lote)
                print(f"Lote de {lote['nombre']} ({lote['codigo']}) no está vencido.")
            return None
        else:
            print("La cola está vacía.")
            return None

    def esta_vacia(self):
        """Verificar si la cola está vacía"""
        return len(self.cola) == 0

    def _producto_vencido(self, lote_producto):
        """Verificar si un producto está vencido"""
        fecha_vencimiento = datetime.strptime(lote_producto['fech_venc'], "%Y-%m-%d")
        return fecha_vencimiento < datetime.now()

    def mostrar_productos(self):
        """Mostrar los productos que están en la cola"""
        if self.esta_vacia():
            print("No hay productos en la cola.")
        else:
            for lote in self.cola:
                print(f"Lote de {lote['nombre']} ({lote['codigo']}), Fecha de vencimiento: {lote['fech_venc']}, Cantidad: {lote['cantidad']}")

    def mostrar_vencidos(self):
        """Mostrar los productos vencidos"""
        if len(self.cola_vencidos) == 0:
            print("No hay productos vencidos.")
        else:
            for lote in self.cola_vencidos:
                print(f"Lote vencido de {lote['nombre']} ({lote['codigo']}), Fecha de vencimiento: {lote['fech_venc']}, Cantidad: {lote['cantidad']}")


In [ ]:
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from datetime import datetime, timedelta
import json

class InventarioApp:
    def __init__(self, root, arbol, cola):
        self.root = root
        self.arbol = arbol
        self.cola = cola

        self.root.title("Gestión de Inventario")
        self.root.geometry("800x600")

        # Crear las pestañas
        self.tab_control = ttk.Notebook(root)
        self.tab_inventario = ttk.Frame(self.tab_control)
        self.tab_agregar_producto = ttk.Frame(self.tab_control)
        self.tab_vencidos = ttk.Frame(self.tab_control)

        self.tab_control.add(self.tab_inventario, text="Inventario Actual")
        self.tab_control.add(self.tab_agregar_producto, text="Agregar Producto")
        self.tab_control.add(self.tab_vencidos, text="Productos Vencidos")
        self.tab_control.pack(expand=1, fill="both")

        # Sección 1: Inventario Actual
        self.inventario_frame = ttk.Frame(self.tab_inventario)
        self.inventario_frame.pack(fill="both", expand=True)

        self.actualizar_btn = ttk.Button(self.inventario_frame, text="Actualizar Inventario", command=self.actualizar_inventario)
        self.actualizar_btn.pack(pady=10)

        self.productos_tree = ttk.Treeview(self.inventario_frame, columns=("Código", "Nombre", "Vencimiento", "Cantidad", "Precio Unitario", "Precio Total"), show="headings")
        self.productos_tree.heading("Código", text="Código")
        self.productos_tree.heading("Nombre", text="Nombre")
        self.productos_tree.heading("Vencimiento", text="Vencimiento")
        self.productos_tree.heading("Cantidad", text="Cantidad")
        self.productos_tree.heading("Precio Unitario", text="Precio Unitario")
        self.productos_tree.heading("Precio Total", text="Precio Total")
        self.productos_tree.pack(fill="both", expand=True)

        # Sección 2: Agregar Producto
        self.agregar_producto_frame = ttk.Frame(self.tab_agregar_producto)
        self.agregar_producto_frame.pack(fill="both", expand=True)

        ttk.Label(self.agregar_producto_frame, text="Nombre del Producto").grid(row=0, column=0, padx=10, pady=10)
        self.nombre_entry = ttk.Entry(self.agregar_producto_frame)
        self.nombre_entry.grid(row=0, column=1)

        ttk.Label(self.agregar_producto_frame, text="Tipo de Producto").grid(row=1, column=0, padx=10, pady=10)
        self.tipo_combobox = ttk.Combobox(self.agregar_producto_frame, values=["Carnes", "Frutas", "Lácteos", "Cereales", "Conservas", "Vegetales"])
        self.tipo_combobox.grid(row=1, column=1)

        ttk.Label(self.agregar_producto_frame, text="Cantidad").grid(row=2, column=0, padx=10, pady=10)
        self.cantidad_entry = ttk.Entry(self.agregar_producto_frame)
        self.cantidad_entry.grid(row=2, column=1)

        ttk.Label(self.agregar_producto_frame, text="Precio Unitario").grid(row=3, column=0, padx=10, pady=10)
        self.precio_entry = ttk.Entry(self.agregar_producto_frame)
        self.precio_entry.grid(row=3, column=1)

        ttk.Label(self.agregar_producto_frame, text="Fecha de Vencimiento").grid(row=4, column=0, padx=10, pady=10)
        self.fecha_calendario = Calendar(self.agregar_producto_frame)
        self.fecha_calendario.grid(row=4, column=1)

        self.agregar_btn = ttk.Button(self.agregar_producto_frame, text="Agregar Producto", command=self.agregar_producto)
        self.agregar_btn.grid(row=5, columnspan=2, pady=20)

        self.mensaje_agregado_label = ttk.Label(self.agregar_producto_frame, text="", font=("Arial", 12))
        self.mensaje_agregado_label.grid(row=6, columnspan=2, pady=10)

        # Sección 3: Productos Vencidos
        self.vencidos_frame = ttk.Frame(self.tab_vencidos)
        self.vencidos_frame.pack(fill="both", expand=True)

        self.vencidos_tree = ttk.Treeview(self.vencidos_frame, columns=("Código", "Nombre", "Vencimiento", "Cantidad", "Precio Total"), show="headings")
        self.vencidos_tree.heading("Código", text="Código")
        self.vencidos_tree.heading("Nombre", text="Nombre")
        self.vencidos_tree.heading("Vencimiento", text="Vencimiento")
        self.vencidos_tree.heading("Cantidad", text="Cantidad")
        self.vencidos_tree.heading("Precio Total", text="Precio Total")
        self.vencidos_tree.pack(fill="both", expand=True)

        # Cargar los productos vencidos (en los últimos 7 días)
        self.mostrar_vencidos()

    def calcular_cantidad_total(self):
        """Calcular la cantidad total de productos en el inventario"""
        cantidad_total = 0
        for tipo in self.arbol.tipos_validos:
            nodo_tipo = self.arbol.raiz.buscar_producto_por_tipo(tipo)
            if nodo_tipo:
                producto_actual = nodo_tipo.producto
                while producto_actual:
                    cantidad_total += producto_actual.cantidad
                    producto_actual = producto_actual.siguiente
        return cantidad_total

    def agregar_producto(self):
        """Agregar un producto al árbol y actualizar el inventario"""
        nombre = self.nombre_entry.get()
        tipo = self.tipo_combobox.get()
        cantidad = int(self.cantidad_entry.get())
        precio = float(self.precio_entry.get())
        fecha_vencimiento_str = self.fecha_calendario.get_date()

        cantidad_total = self.calcular_cantidad_total()

        # Verificar si agregar el nuevo producto superaría el límite de 10,000 productos
        if cantidad_total + cantidad > 10000:
            self.mensaje_agregado_label.config(text="Error: El inventario no puede superar los 10,000 productos.", foreground="red")
            return

        fecha_vencimiento = datetime.strptime(fecha_vencimiento_str, "%m/%d/%y").strftime("%Y-%m-%d")

        # Crear producto y agregar al árbol
        producto = Producto(
            nombre=nombre,
            tipo=tipo,
            cantidad=cantidad,
            precio=precio,
            fecha_vencimiento=fecha_vencimiento
        )
        self.arbol.agregar_producto_a_tipo(tipo, producto)
        self.mostrar_inventario() 
        self.mensaje_agregado_label.config(text="Producto agregado correctamente.", foreground="green")

    def mostrar_inventario(self):
        """Mostrar productos no vencidos en el inventario"""
        # Limpiar el árbol actual
        for item in self.productos_tree.get_children():
            self.productos_tree.delete(item)

        # Mostrar productos divididos por tipo
        for tipo in self.arbol.tipos_validos:
            nodo_tipo = self.arbol.raiz.buscar_producto_por_tipo(tipo)
            if nodo_tipo:
                producto_actual = nodo_tipo.producto
                while producto_actual:
                    if not self.cola._producto_vencido({'fech_venc': producto_actual.fecha_vencimiento.strftime("%Y-%m-%d")}):
                        self.productos_tree.insert("", "end", values=(
                            producto_actual.codigo,
                            producto_actual.nombre,
                            producto_actual.fecha_vencimiento.strftime("%Y-%m-%d"),
                            producto_actual.cantidad,
                            f"{producto_actual.precio:.2f}",
                            f"{producto_actual.precio_total:.2f}"
                        ))
                    producto_actual = producto_actual.siguiente

    def actualizar_inventario(self):
        """Actualizar el inventario mostrando solo los productos no vencidos"""
        self.cola.retirar() 
        self.mostrar_inventario()

    def mostrar_vencidos(self):
        """Mostrar productos vencidos dentro de los últimos 7 días"""
        
        for item in self.vencidos_tree.get_children():
            self.vencidos_tree.delete(item)

        # Buscar y mostrar productos vencidos
        for tipo in self.arbol.tipos_validos:
            nodo_tipo = self.arbol.raiz.buscar_producto_por_tipo(tipo)
            if nodo_tipo:
                producto_actual = nodo_tipo.producto
                while producto_actual:
                    if self.cola._producto_vencido({'fech_venc': producto_actual.fecha_vencimiento.strftime("%Y-%m-%d")}) and \
                            producto_actual.fecha_vencimiento >= datetime.now() - timedelta(weeks=1):
                        self.vencidos_tree.insert("", "end", values=(
                            producto_actual.codigo,
                            producto_actual.nombre,
                            producto_actual.fecha_vencimiento.strftime("%Y-%m-%d"),
                            producto_actual.cantidad,
                            f"{producto_actual.precio_total:.2f}"
                        ))
                    producto_actual = producto_actual.siguiente


In [7]:
# Inicializar objetos Arbol y Cola
arbol_productos = Arbol("Inventario")
cola_productos = Cola()

In [ ]:
# Función para cargar datos desde archivo
def cargar_datos_desde_archivo(nombre_archivo, arbol):
    with open(nombre_archivo, 'r', encoding='utf-8') as archivo: 
        lineas = archivo.readlines()
        for linea in lineas:
            datos = json.loads(linea.strip())
            producto = Producto(
                nombre=datos["nombre"],
                fecha_vencimiento=datos.get("fech_venc"),
                tipo=datos["tipo"],  # Carnes, Frutas, Cereales, Conservas, Lácteos
                cantidad=int(datos["cantidad"]),
                precio=float(datos["precio"])  
            )
            # Verificar si el tipo de producto ya existe, y si no, agregarlo
            nodo_tipo = arbol.raiz.buscar_producto_por_tipo(producto.tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(producto.tipo)
                arbol.agregar_tipo(nodo_tipo)
            arbol.agregar_producto_a_tipo(producto.tipo, producto)

In [9]:
cargar_datos_desde_archivo('inventario.txt', arbol_productos)

In [10]:
imprimir_productos_por_tipo(arbol_productos, "Frutas")

Productos de tipo 'Frutas':
Código: E2AHAZ, Nombre: Manzana, Vencimiento: 2024-12-01 00:00:00, Cantidad: 50, Precio unitario: 0.5, Precio total: 25.0
Código: JFEAON, Nombre: Pera, Vencimiento: 2024-11-30 00:00:00, Cantidad: 60, Precio unitario: 0.7, Precio total: 42.0


In [11]:
agregar_producto(arbol_productos, "Pescado", "2024-11-24", "Carnes", 100, 0.6)

Producto 'Pescado' agregado correctamente.


In [ ]:
# Crear y lanzar la interfaz
root = tk.Tk()
app = InventarioApp(root, arbol_productos, cola_productos)
root.mainloop()

La cola está vacía.
La cola está vacía.
